In [633]:
import math
import numpy as np
from cvxopt import matrix, glpk
from prettytable import PrettyTable

import sys

np.set_printoptions(threshold=sys.maxsize)

In [634]:
need = [
    {"morning": [8, 8], "evening": [7, 6]},  # monday
    {"morning": [6, 5], "evening": [4, 5]},  # tuesday
    {"morning": [6, 5], "evening": [4, 4]},  # wednesday
    {"morning": [5, 6], "evening": [6, 7]},  # thursday
    {"morning": [7, 8], "evening": [8, 9]},  # friday
    {"morning": [9, 8], "evening": [7, 6]},  # saturday
    {"morning": [6, 5], "evening": [4, 4]},  # sunday
]
daysToWork = 5
regularMonthPayment = 51200
temporaryHourPayment = 380

In [635]:
regularPeopleByDay = [
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    #
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
]
regularNeedByDay = []
for i in range(len(regularPeopleByDay)):
    regularPeopleByDay[i] = np.concatenate(
        (np.array(regularPeopleByDay[i]), np.zeros(len(regularPeopleByDay) * 2))
    )
    regularNeedByDay.append(
        min(need[i]["morning"] if i < len(need) else need[i - len(need)]["evening"])
    )
regularPeopleByDay = np.array(regularPeopleByDay) * -1
regularNeedByDay = np.array(regularNeedByDay) * -1

In [636]:
temporaryPeopleByDay = np.array([])
temporaryNeedByDay = []
day = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]
for i in range(len(need)):
    for j in range(len(day)):
        weekDayRow = np.concatenate(
            (
                np.zeros(len(regularPeopleByDay) + i * 4),
                np.array(day[j]),
                np.zeros((len(need) - 1 - i) * 4),
            )
        )
        temporaryPeopleByDay = (
            np.copy(weekDayRow)
            if (i == 0 and j == 0)
            else np.vstack((temporaryPeopleByDay, weekDayRow))
        )
        weekDayNeed = need[i]["morning"] if j < 2 else need[i]["evening"]
        temporaryNeedByDay.append(weekDayNeed[j % 2] - min(weekDayNeed))
temporaryPeopleByDay *= -1
temporaryNeedByDay = np.array(temporaryNeedByDay) * -1

In [637]:
notNegativeAmount = np.array([])
amountOfX = len(regularPeopleByDay[0])
for i in range(amountOfX):
    row = np.concatenate((np.zeros(i), np.array([-1]), np.zeros(amountOfX - i - 1)))
    notNegativeAmount = (
        np.copy(row) if (i == 0) else np.vstack((notNegativeAmount, row))
    )

In [638]:
regularHourPayment = regularMonthPayment / 4 / 40
cost = np.concatenate(
    (
        np.full(len(regularPeopleByDay), regularHourPayment),
        np.full(len(temporaryPeopleByDay), temporaryHourPayment),
    )
)

In [639]:
targetFunction = matrix(cost, tc='d').T
inequalityLeft = matrix(
    [
        [matrix(regularPeopleByDay).T],
        [matrix(temporaryPeopleByDay).T],
        [matrix(notNegativeAmount).T],
    ], tc='d'
).T
inequalityRight = matrix(
    [
        [matrix(regularNeedByDay).T],
        [matrix(temporaryNeedByDay).T],
        [matrix(np.zeros(len(notNegativeAmount))).T],
    ], tc='d'
).T

In [640]:
(status, x) = glpk.ilp(
    c=targetFunction, G=inequalityLeft, h=inequalityRight, I=set(range(len(targetFunction)))
)

In [641]:
print(status)
print(x)
for i in range(len(need) - 1):
    for j in range(i*len(need), (i+1)*len(need)):
        print(x[j], end=' ')
        if j == 13:
            print('')
    print()

optimal
1.0 2.0 0.0 1.0 3.0 2.0 1.0 
2.0 2.0 0.0 4.0 0.0 0.0 0.0 

0.0 0.0 1.0 0.0 1.0 0.0 0.0 
1.0 1.0 0.0 0.0 0.0 0.0 1.0 
0.0 1.0 0.0 1.0 0.0 1.0 1.0 
0.0 1.0 0.0 1.0 0.0 0.0 0.0 


In [ ]:
weekDays = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
def printResult(resultStatus, resultArray):
    print(f"Статус: {resultStatus}")
    if resultStatus == 'optimal':
        print("График выхода работников:")
        table = PrettyTable(["День недели", "7:00 - 15:00", "15:00 - 23:00"])
        table.add_row([weekDays, ])
    else:
        print("Оптимальное решение не было найдено.")

In [642]:
print(regularNeedByDay)

[-8 -5 -5 -5 -7 -8 -5 -6 -4 -4 -6 -8 -6 -4]
